# Visualize computational graphs

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
%load_ext line_profiler
from torch.autograd import profiler
from torchviz import make_dot

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge
from madminer.utils.ml.ratio_trainer import train_ratio_model
from madminer.utils.ml.models.ratio import ParameterizedRatioEstimator


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.WARNING
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [14]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'
figure_dir = base_dir + 'figures/wgamma/'

## Calculate computational graphs

In [21]:
ml = MLForge()

batch_size = 6

loss_rolr, params_rolr = ml.train(
    method='rolr',
    x_filename=sample_dir + 'train_ratio_tight/x_train_0.npy',
    theta0_filename=sample_dir + 'train_ratio_tight/theta0_train_0.npy',
    r_xz_filename=sample_dir + 'train_ratio_tight/r_xz_train_0.npy',
    y_filename=sample_dir + 'train_ratio_tight/y_train_0.npy',
    limit_samplesize=batch_size,
    batch_size=batch_size,
    return_first_loss=True
)

loss_rascal, params_rascal = ml.train(
    method='rascal',
    x_filename=sample_dir + 'train_ratio_tight/x_train_0.npy',
    theta0_filename=sample_dir + 'train_ratio_tight/theta0_train_0.npy',
    t_xz0_filename=sample_dir + 'train_ratio_tight/t_xz_train_0.npy',
    r_xz_filename=sample_dir + 'train_ratio_tight/r_xz_train_0.npy',
    y_filename=sample_dir + 'train_ratio_tight/y_train_0.npy',
    limit_samplesize=batch_size,
    batch_size=batch_size,
    return_first_loss=True
)

loss_nde, params_nde = ml.train(
    method='nde',
    x_filename=sample_dir + 'train_scandal_tight/x_train_0.npy',
    theta0_filename=sample_dir + 'train_scandal_tight/theta_train_0.npy',
    limit_samplesize=batch_size,
    batch_size=batch_size,
    return_first_loss=True,
    maf_mog_n_components=4,
    maf_n_mades=2,
    n_hidden=(100,),
    
)

loss_scandal, params_scandal = ml.train(
    method='scandal',
    x_filename=sample_dir + 'train_scandal_tight/x_train_0.npy',
    theta0_filename=sample_dir + 'train_scandal_tight/theta_train_0.npy',
    t_xz0_filename=sample_dir + 'train_scandal_tight/t_xz_train_0.npy',
    limit_samplesize=batch_size,
    batch_size=batch_size,
    return_first_loss=True,
    maf_mog_n_components=4,
    maf_n_mades=2,
    n_hidden=(100,),
)

## ROLR graph

In [22]:
dot = make_dot(loss_rolr, params=params_rolr)
dot.render(figure_dir + 'graphs/rolr')

'/Users/johannbrehmer/work/projects/madminer/diboson_mining/figures/wgamma/graphs/rolr.pdf'

## RASCAL graph

In [23]:
dot = make_dot(loss_rascal, params=params_rascal)
dot.render(figure_dir + 'graphs/rascal')

'/Users/johannbrehmer/work/projects/madminer/diboson_mining/figures/wgamma/graphs/rascal.pdf'

## NDE graph

In [24]:
dot = make_dot(loss_nde, params=params_nde)
dot.render(figure_dir + 'graphs/nde')

'/Users/johannbrehmer/work/projects/madminer/diboson_mining/figures/wgamma/graphs/nde.pdf'

## SCANDAL graph

In [25]:
dot = make_dot(loss_scandal, params=params_scandal)
dot.render(figure_dir + 'graphs/scandal')

'/Users/johannbrehmer/work/projects/madminer/diboson_mining/figures/wgamma/graphs/scandal.pdf'